# WARNING: Although we intended to use the DESDEO library, we found that the Pymoo library provided the flexibility needed for this problem. DESDEO could not handle discrete integer values as well as Pymoo could. The constraints were more often violated in DESDEO and it had trouble finding feasible solutions. The final code is available in CargoOpt_Pymoo.ipynb
We kept the notebook to show our desdeo attempt and potentially implement it in the future.

# Case 2: Cargo Ship
The ship MS Leiden is traveling on a route involving five harbors in north-
ern Europe: Rotterdam, Hamburg, Kiel, Aarhus, and Copenhagen. You have
been asked to help the Captain with the loading and unloading plan (for the
destination harbors) that produces

- even and well-balanced solution
- a solution that is easy to unload (so that containers that are earlier un-loaded are not beneath containers that are later unloaded)
- solutions that can integrate as many containers as possible

The following details about the problem are provided:
1. The ship is now docked in Rotterdam. After visiting Rotterdam, the ship
will be traveling to Hamburg, Aarhus, and Copenhagen, in that order. The
containers destined for Rotterdam have been unloaded and the containers
destined for the remaining three harbors have to be loaded on the ship.
2. MS Leiden is a rather small container ship. It has eight bays, three tiers
and four rows. The layout of the ship is shown in the figure below.
3. Containers should be loaded such that a container that has to be unloaded
earlier should be placed in a higher position.
4. Each cell in the above figure is able to hold a forty-foot container. That
is, the ship has room for 8 × 3 × 4 = 96 forty-foot containers.
5. A forty-foot container can be placed on-top of another forty-foot container,
but not on top of an empty cell. We assume that only forty-foot containers
are loaded on the ship. Each container has a destination port.
6. Each container $i$ has a certain weight $w_i$. If a container is much heavier
than another $i$ container $j$, say $w_i − w_j > δ_w$, then it is not allowed to
place $w_i$ on top of $w_j$ .
7. The containers should be balanced in a way that is evenly distributed. The
longitudinal center of gravity should be as close as possible to the middle
of the container section of the ship. Secondly, the latitudinal center of
gravity should be as much to the middle as possible. Note that the center
of gravity can be computed as the weighted sum of the centroids of the
containers, where the weights are the total weight of the containers.

In [2]:
from desdeo_problem import variable_builder, ScalarObjective, ScalarConstraint, MOProblem
from desdeo_problem.testproblems.TestProblems import test_problem_builder
from desdeo_emo.population import Population

from desdeo_emo.EAs import NSGAIII, PPGA
from desdeo_mcdm.interactive.NIMBUS import NIMBUS

import numpy as np

# desdeo.probdefs.Problem.__abstractmethods__ = set()
# desdeo docs: https://desdeo.readthedocs.io/en/latest/

![title](Docs/cargo1.png)

![title](Docs/cargo2.png)

![title](Docs/cargo3.png)

## Code

Balance score can be calculated as:
The containers should be balanced in a way that is evenly distributed. The
longitudinal center of gravity should be as close as possible to the middle
of the container section of the ship. Secondly, the latitudinal center of
gravity should be as much to the middle as possible. Note that the center
of gravity can be computed as the weighted sum of the centroids of the
containers, where the weights are the total weight of the containers.

In [3]:
# Create DATA
# n = np.random.randint(10, 96)
n = 50
h = np.random.randint(1, 4, n)
w = np.random.randint(2000, 4000, n)
DATA = np.array(list(zip(h,w)))

HARBORS = np.unique(DATA[:,0])

In [4]:
print(DATA)

[[   2 2172]
 [   2 2151]
 [   1 3057]
 [   2 3463]
 [   3 3363]
 [   1 3571]
 [   2 2290]
 [   2 3201]
 [   3 2527]
 [   2 2975]
 [   3 3604]
 [   1 3330]
 [   3 2360]
 [   2 3479]
 [   2 2613]
 [   2 2372]
 [   3 3126]
 [   1 2316]
 [   3 2488]
 [   1 3872]
 [   3 2544]
 [   2 2133]
 [   3 3196]
 [   2 3951]
 [   1 2662]
 [   1 3463]
 [   1 3222]
 [   3 3075]
 [   3 3712]
 [   1 3214]
 [   1 3038]
 [   2 2779]
 [   1 3945]
 [   1 3838]
 [   2 2738]
 [   3 2240]
 [   1 3979]
 [   3 3495]
 [   2 3987]
 [   2 3804]
 [   2 3187]
 [   2 3335]
 [   2 3935]
 [   2 3497]
 [   2 3912]
 [   1 2690]
 [   2 3856]
 [   3 2877]
 [   3 3228]
 [   1 3399]]


In [5]:
DATA = np.array([[1, 2873],[1, 2116],[3, 2264],[3, 2215],[1, 3160],[1, 2493],[1, 3670],[3, 2771],[1, 2567],[1, 3064]])
HARBORS = np.unique(DATA[:,0])
n = len(DATA)

### Helper functions

In [6]:
def remove_harbor_containers(configuration, destination):
    # [2,1,2] -> remove number -> [2,2] -> length of 3 -> [2,2,0]
    for bay in range(configuration.shape[0]):
        for row in range(configuration.shape[2]):
            stack = configuration[bay, :, row, :]
            if destination in stack[:,0]:
                stack = stack[stack[:,0] != destination]
                stack = np.append(stack, np.zeros((3-len(stack),2)), axis=0)
                configuration[bay, :, row, :] = stack
    return configuration

def positions_to_matrix(positions):
    global temp_pos
    temp_pos = positions.copy()
    configuration = np.zeros((8,3,4,2), dtype=int)

    for i, position in enumerate(positions):

        layer = int(position // 32)
        position = position % 32
        bay = int(position // 4)
        row = int(position % 4)
        
        configuration[bay, layer, row, :] = DATA[i]

    return configuration

def create_population(pop_size):

    sorted_data = np.flip(DATA[:, 1].argsort(), axis=0)

    population = []
    for _ in range(pop_size):

        individual = np.zeros(len(DATA))
        for idx in sorted_data:

            while True:

                rand_position = np.random.randint(1, 33)

                positions = set(np.arange(rand_position, 97, 32)) - set(individual)

                if len(positions) > 0: 
                    individual[idx] = min(positions)
                    break
        
        population.append(individual-1)

    return np.array(population)

### Objective 1: Stability

In [7]:
def sum_tiers(configuration):

    only_weights = configuration[:, :, :, 1]
    summed_configuration = np.sum(only_weights, axis=1)

    return summed_configuration

def get_longitudinal_center(summed_configuration):

    weights = np.sum(summed_configuration, axis=1)
    positions = np.arange(1, 9)

    sum_weights = np.sum(weights)
    if sum_weights != 0:
        return np.sum(weights * positions) / sum_weights
    else:
        return 1

def get_lattitudinal_center(summed_configuration):

    weights = np.sum(summed_configuration, axis=0)
    positions = np.arange(1, 5)

    sum_weights = np.sum(weights)
    if sum_weights != 0:
        return np.sum(weights * positions) / sum_weights
    else:
        return 1


def calculate_centers(configuration):

    summed_configuration = sum_tiers(configuration)

    longitudinal_center = get_longitudinal_center(summed_configuration)
    lattitudinal_center = get_lattitudinal_center(summed_configuration)

    return longitudinal_center, lattitudinal_center

def get_score(configuration):

    longitudinal_center, lattitudinal_center = calculate_centers(configuration)

    long_error, latt_error = abs(longitudinal_center - 4.5), abs(lattitudinal_center - 2.5)

    return long_error + latt_error

def calculate_total_stability(candidate_solutions):

    stability_scores = []

    for candidate in candidate_solutions:

        configuration = positions_to_matrix(candidate)

        stability_score = 0
        for i in HARBORS:
            stability_score += get_score(configuration)
            
            if i < 3:
                configuration = remove_harbor_containers(configuration, i)
        
        stability_scores.append(stability_score)
    # print('Stability scores:',stability_scores)
    return stability_scores

In [8]:
configuration = positions_to_matrix([34.62940047, 70.55500473, 17.87867337, 71.48437768, 71.53891922, 17.35121299, 40.18080825, 27.60435269, 27.15240865, 20.19477824])

stability_score = 0

stability_score += get_score(configuration)
    
configuration2 = remove_harbor_containers(configuration, 1)

summed_configuration = sum_tiers(configuration2)
print(summed_configuration)
# longitudinal_center, lattitudinal_center = calculate_centers(configuration2)

[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]


### Objective 2: Unloading time

In [9]:
def get_unloading_time(candidate_solutions):

    unloading_times = []
    
    for candidate in candidate_solutions:

        configuration = positions_to_matrix(candidate)
        only_harbors = configuration[:, :, :, 0]

        unloading_time = 0
        for i in range(8):
            for j in range(4):
                stack = only_harbors[i, :, j]
                stack = stack[stack != 0]
                if len(stack) > 1:
                    if tuple(stack) == (2,1,3):
                        return 4
                    for layer in range(len(stack)-1):
                        if stack[layer] == 1:
                            if stack[layer+1] != 1:
                                unloading_time += 2
                            if len(stack[layer:]) > 2:
                                if stack[layer+2] != 1 and stack[layer+1] != 1:
                                    unloading_time += 2
                        elif stack[layer] == 2:
                            if stack[layer+1] == 3:
                                unloading_time += 2
                            if len(stack[layer:]) > 2:
                                if stack[layer+2] == 3 and stack[layer+1] == 3:
                                    unloading_time += 2
        unloading_times.append(unloading_time/n)
    # print('Unloading times:',unloading_times)
    return unloading_times

In [10]:
def func(stack):
    unloading_time = 0
    if tuple(stack) == (2,1,3):
        return 4
    for layer in range(len(stack)-1):
        if stack[layer] == 1:
            if stack[layer+1] != 1:
                unloading_time += 2
            if len(stack[layer:]) > 2:
                if stack[layer+2] != 1 and stack[layer+1] != 1:
                    unloading_time += 2
        elif stack[layer] == 2:
            if stack[layer+1] == 3:
                unloading_time += 2
            if len(stack[layer:]) > 2:
                if stack[layer+2] == 3 and stack[layer+1] == 3:
                    unloading_time += 2
    return unloading_time

In [11]:
# Constants
delta_weight = 2000

# OPTION 1
# GENERATE A VALID SOLUTION TO START WITH
# SORTING BY WEIGHT AND PLACING FROM BOTTOM TO TOP

# OPTION 2
# CHANGE CONSTRAINTS TO RETURN MORE NEGATIVE VALUES IF CONSTRAINTS ARE BREACHED
# "MORE".


# Constraints
# 1. Containers can't be placed on the same position
# 2. Containers can not levitate
# 3. Containers can only weigh delta_weight kg more than the container below it
# 
# Objectives
# 1. Minimize the long_error + latt_error (maximize the stability of the ship)
# 2. Minimize the unloading time of the ship

# Create variables
var_names = ["c"+str(id) for id in range(len(DATA))]

initial_values = [pos for pos in range(len(DATA))]
lower_bounds = [0] * len(DATA)
upper_bounds = [95.9999] * len(DATA)

variables = variable_builder(var_names, initial_values, lower_bounds, upper_bounds)

# Create objectives
objectives = [
    ScalarObjective(name="Stability", evaluator=calculate_total_stability, maximize=False),
    ScalarObjective(name="Unloading time", evaluator=get_unloading_time, maximize=False)
    ]

# Create constraints
def overlapping_eval(candidates, fitnesses):

    constraint_values = []
    for positions in candidates:
        positions = [int(pos) for pos in positions]
        
        if n != len(set(positions)):
            constraint_values.append(-1)
        else:
            constraint_values.append(1)

    return constraint_values
    

def overlapping_eval2(candidates, fitnesses):

    constraint_values = []
    for positions in candidates:
        positions = [int(pos) for pos in positions]
        
        if n != len(set(positions)):
            constraint_values.append(-1 * (n - len(set(positions))))
        else:
            constraint_values.append(1)

    return constraint_values


def levitating_eval(candidates, fitnesses):

    constraint_values = []
    for positions in candidates:

        positions = [int(pos) for pos in positions]
        
        breach = False

        for pos in positions:

            if pos < 32:
                continue
            else:
                # pos_below = np.arange(pos-32, -1, -32)
                pos_below = pos - 32
                if not pos_below in positions:
                    breach = True
                    break
        
        if breach:
            constraint_values.append(-1)
        else:
            constraint_values.append(1)
            
    return constraint_values


def levitating_eval2(candidates, fitnesses):

    constraint_values = []
    for positions in candidates:

        positions = [int(pos) for pos in positions]
        
        breachness = 0

        for pos in positions:

            if pos < 32:
                continue
            else:
                # pos_below = np.arange(pos-32, -1, -32)
                pos_below = pos - 32
                if not pos_below in positions:
                    print('We have:',pos,' but we dont have:', pos_below)
                    breachness -= 1
        
        if breachness < 0:
            constraint_values.append(breachness)
        else:
            constraint_values.append(1)
            
    return constraint_values


def weight_eval(candidates, fitnesses):

    constraint_values = []
    for positions in candidates:

        positions = [int(pos) for pos in positions]

        pos_weights = list(zip(positions, DATA[:,1]))

        breach = False

        for pos, weight in pos_weights:
            if pos < 32:
                continue
            else:
                pos_below = pos - 32
                try:
                    weight_below = pos_weights[positions.index(pos_below)][1]
                    if weight > weight_below + delta_weight:
                        breach = True
                        break
                except:
                    continue
        if breach:
            constraint_values.append(-1)
        else:
            constraint_values.append(1)

    return constraint_values


def weight_eval2(candidates, fitnesses):

    constraint_values = []
    for positions in candidates:

        positions = [int(pos) for pos in positions]

        pos_weights = list(zip(positions, DATA[:,1]))
        breachness = 0
        for pos, weight in pos_weights:
            if pos < 32:
                continue
            else:
                pos_below = pos - 32
                try:
                    weight_below = pos_weights[positions.index(pos_below)][1]
                    if weight > weight_below + delta_weight:
                        breachness += weight_below + delta_weight - weight
                except:
                    continue
        if breachness < 0:
            constraint_values.append(breachness)            
        else:
            constraint_values.append(1)
    return constraint_values





In [12]:
constraints = [
    ScalarConstraint(name="Overlapping", n_decision_vars=len(DATA), n_objective_funs=2, evaluator=overlapping_eval2),
    ScalarConstraint(name="Levitating", n_decision_vars=len(DATA), n_objective_funs=2, evaluator=levitating_eval2),
    ScalarConstraint(name="Weight", n_decision_vars=len(DATA), n_objective_funs=2, evaluator=weight_eval2)
    ]

problem = MOProblem(objectives, variables, constraints)

method = 'NIMBUS'

if method == 'EA':
    # Evolutionary Algorithm
    # Create an algorithm
    evolver = NSGAIII(
        problem=problem,
        n_iterations=20,
        n_gen_per_iter=100
        )
    
    pop_size = 99
    new_population = create_population(pop_size)
    print(new_population)
    initial_population = Population(
        problem=None,
        pop_size=1
        ).add(
            offsprings=new_population
        )


    # Run the algorithm
    iter = 0
    while evolver.continue_evolution():
        print('Generation:',iter)
        print('Average stability:',np.average(evolver.population.fitness[:,0]))
        print('Average unloading time:',np.average(evolver.population.fitness[:,1]),'\n')
        evolver.iterate()
        iter += 1

elif method == 'NIMBUS':

    method = NIMBUS(problem, "scipy_de")

    classification_request, plot_request = method.start()

[[22. 19. 27. 25. 12.  6. 28.  4. 10. 30.]
 [13. 15. 43. 45. 26.  3. 11.  0. 10. 23.]
 [29. 43. 30. 11.  3. 57. 25.  1.  0. 13.]
 [ 3. 28.  8.  7. 11. 18. 16.  2. 12. 20.]
 [26.  6.  4.  1. 12. 17. 24. 20. 23. 15.]
 [ 8. 24. 18. 11.  7.  4.  2. 19. 61. 29.]
 [ 6. 13. 14. 18. 17. 22.  8. 15. 19.  5.]
 [ 2.  9. 14. 27. 28. 22. 26.  5. 15.  7.]
 [23. 34. 27.  2. 16.  0.  4. 21. 15.  8.]
 [26. 19. 30. 31.  8.  4. 27. 15.  3. 24.]
 [26. 25.  8.  3.  1.  7.  0. 16.  2.  5.]
 [ 5. 62. 41. 13. 27. 25.  2.  9.  1. 30.]
 [35. 18.  5. 27. 24. 56.  9. 25. 31.  3.]
 [30. 33. 14. 23. 20. 94.  1. 62.  5.  2.]
 [16. 20.  6. 18.  4. 27.  9.  8. 25. 41.]
 [29. 16. 21. 15. 30. 14. 11. 12.  3. 19.]
 [29. 28. 26. 36. 21.  2.  7.  4. 15. 24.]
 [ 6. 28. 11. 12. 22. 30.  9. 54.  1. 21.]
 [19. 30. 46. 10. 28. 60. 14. 29. 37.  5.]
 [ 6. 22. 10. 32.  2.  0. 15. 17. 31. 25.]
 [11. 30. 24.  4.  6. 22. 28.  5. 38.  7.]
 [23. 16.  2. 22. 25.  1. 15. 21. 30. 13.]
 [ 2. 27. 17. 34.  5.  1.  8.  7. 30.  4.]
 [19. 33. 1

AttributeError: 'NoneType' object has no attribute 'n_of_constraints'

In [ ]:
pop_size = 100
initial_population = create_population(pop_size)

print(overlapping_eval2(initial_population,[]))
print(levitating_eval2(initial_population,[]))
print(weight_eval2(initial_population,[]))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
if method == 'EA':
    print(evolver.population.constraint)
    print(evolver.population.fitness)
elif method == 'NIMBUS':
    print(classification_request.content["objective_values"])

[[-14. -18.   1.]
 [-10. -22.   1.]
 [-12. -19.   1.]
 [-12. -19.   1.]
 [-12. -19.   1.]
 [-12. -18.   1.]
 [-12. -20.   1.]
 [-12. -19.   1.]
 [-12. -19.   1.]
 [-10. -20.   1.]
 [-12. -19.   1.]
 [-12. -20.   1.]
 [-12. -19.   1.]
 [-12. -20.   1.]
 [-12. -20.   1.]
 [-12. -18.   1.]
 [-12. -20.   1.]
 [-12. -20.   1.]
 [-12. -18.   1.]
 [-12. -20.   1.]
 [-12. -19.   1.]
 [-12. -19.   1.]
 [-12. -19.   1.]
 [-12. -18.   1.]
 [-12. -19.   1.]
 [-12. -20.   1.]
 [-12. -20.   1.]
 [-12. -18.   1.]
 [-12. -20.   1.]
 [-12. -20.   1.]
 [-12. -20.   1.]
 [-12. -20.   1.]
 [-12. -19.   1.]
 [-12. -18.   1.]
 [-12. -20.   1.]
 [-12. -20.   1.]
 [-12. -17.   1.]
 [-12. -20.   1.]
 [-12. -20.   1.]
 [-12. -19.   1.]
 [-12. -19.   1.]
 [-12. -19.   1.]
 [-12. -19.   1.]
 [-12. -19.   1.]
 [-12. -20.   1.]
 [-12. -19.   1.]
 [-12. -19.   1.]
 [-12. -20.   1.]
 [-12. -20.   1.]
 [-12. -20.   1.]
 [-12. -18.   1.]
 [-12. -19.   1.]
 [-12. -20.   1.]
 [-12. -19.   1.]
 [-12. -19.   1.]
 [-12. -19